In [167]:
import requests
import pandas as pd

api_token = '8a0b92957b57d4eea9cdf73c4d7436c09c82a38865b84e5b2304b7b5a0202b92'
coins_endpoint = f'https://whattomine.com/api/v1/coins?api_token={api_token}'
coins_data = requests.get(coins_endpoint).json()

coins = [ 'BTC', 'BCH', 'DOGE', 'LTC', 'KAS', 'RVN' ]
columns = [ 'block_time', 'block_reward', 'exchange_rate_btc', 'algorithm', 'nethash_eh_s' ] 
index = []
data = []
btc_exchange_rate = 0

def get_exchange_rate(tag, price):
    if tag == 'BTC':
        global btc_exchange_rate
        btc_exchange_rate = price
        return 1
    return price

for c in coins_data:
    if c['tag'] not in coins:
        continue
    index.append(c['tag'])
    data.append([ float(c['block_time']), c['block_reward'],  get_exchange_rate(c['tag'], c['exchanges'][0]['price']), c['algorithm'], c['nethash'] / 1e18 ])

global_df = pd.DataFrame(data, index=index, columns=columns)
global_df['blocks_per_day'] = 86400 / global_df.block_time
global_df['block_reward_btc'] = global_df.block_reward * global_df.exchange_rate_btc
global_df['daily_emmisions_btc'] = global_df.blocks_per_day * global_df.block_reward_btc
global_df['index_price_btc_eh_day'] = global_df.daily_emmisions_btc / global_df.nethash_eh_s # BTC / EH / day
# global_df.loc['DOGE+LTC', ['daily_emmisions_btc']] = global_df.loc['DOGE', ['daily_emmisions_btc']] + global_df.loc['LTC', ['daily_emmisions_btc']]
print('Global blockaing data:')
# print(df)
global_df

Global blockaing data:


,block_time,block_reward,exchange_rate_btc,algorithm,nethash_eh_s,blocks_per_day,block_reward_btc,daily_emmisions_btc,index_price_btc_eh_day
BCH,595.0000,3.125000,3.481312e-03,SHA-256,2.849742,145.210084,0.010879,1.579755,0.554350
BTC,652.0000,3.156034,1.000000e+00,SHA-256,752.062414,132.515337,3.156034,418.222897,0.556101
DOGE,63.0000,10000.000000,2.693173e-06,Scrypt,0.002678,1371.428571,0.026932,36.934947,13789.721428
KAS,0.9998,65.728645,9.308327e-07,kHeavyHash,1.369009,86417.283457,0.000061,5.287215,3.862074
LTC,141.0000,6.250000,1.322000e-03,Scrypt,0.002323,612.765957,0.008263,5.062979,2179.125930
RVN,59.0000,2500.000000,1.400000e-07,KawPow,0.000007,1464.406780,0.000350,0.512542,78494.815591


In [168]:
import requests
import pandas as pd

algorithms_response = requests.get('https://api2.nicehash.com/main/api/v2/public/simplemultialgo/info')
multialgo_data = algorithms_response.json()
algorithms = [ 'SCRYPT', 'SHA256ASICBOOST', 'KAWPOW', 'KHEAVYHASH']
columns = ['paying', 'speed']
index = []
data = []
for a in multialgo_data['miningAlgorithms']:
    algo = a['algorithm']
    if algo not in algorithms:
        continue
    index.append(algo)
    data.append([a['paying'], a['speed']])
algo_data_df = pd.DataFrame(data, index=index, columns=columns)

factors_response = requests.get('https://api2.nicehash.com/main/api/v2/mining/algorithms')
factors_data = factors_response.json()
columns = ['market_factor', 'mining_factor', 'price_factor']
index = []
data = []
for f in factors_data['miningAlgorithms']:
    algo = f['algorithm']
    if algo not in algorithms:
        continue
    index.append(algo)
    data.append([float(f['marketFactor']), float(f['miningFactor']), float(f['priceFactor'])])
factors_data_df = pd.DataFrame(data, index=index, columns=columns)
factors_data_df

sats2btc = 1e8
nh_market_df = pd.DataFrame(index=index)
nh_market_df['hashrate_eh_s'] = algo_data_df['speed'] * factors_data_df['mining_factor'] / 1e18 # EH / s
nh_market_df['paying_btc_eh_day'] = algo_data_df['paying'] / sats2btc  * 1e18 # BTC/EH/day
nh_market_df['paying_btc_eh_day_$'] = (nh_market_df.paying_btc_eh_day * btc_exchange_rate).map('${:,.2f}'.format)
nh_market_df['daily_consumption_btc'] = nh_market_df.hashrate_eh_s * nh_market_df.paying_btc_eh_day
print('Nicehash marketplace data:')
# print(market_data_df)
nh_market_df

Nicehash marketplace data:


,hashrate_eh_s,paying_btc_eh_day,paying_btc_eh_day_$,daily_consumption_btc
SCRYPT,1.294292e-04,16333.926636,"$1,596,948,733.17",2.114087
SHA256ASICBOOST,6.889826e+00,0.540129,"$52,807.82",3.721398
KAWPOW,7.456578e-07,81504.187063,"$7,968,568,194.17",0.060774
KHEAVYHASH,5.448271e-02,4.312793,"$421,656.64",0.234973


In [169]:
max_market_utilisation = 0.76

nh_limits_df = pd.DataFrame(index=nh_market_df.index)
nh_limits_df['max_hashrate_eh_s'] = nh_market_df.hashrate_eh_s * max_market_utilisation
nh_limits_df['max_daily_consumption_btc'] = nh_market_df.daily_consumption_btc * max_market_utilisation
nh_limits_df['max_hourly_consumption_btc'] = nh_limits_df.max_daily_consumption_btc / 24
nh_limits_df['max_hourly_consumption_$'] = (nh_limits_df.max_hourly_consumption_btc * btc_exchange_rate).map('${:,.2f}'.format)
nh_limits_df

,max_hashrate_eh_s,max_daily_consumption_btc,max_hourly_consumption_btc,max_hourly_consumption_$
SCRYPT,9.836617e-05,1.606706,0.066946,"$6,545.24"
SHA256ASICBOOST,5.236268e+00,2.828262,0.117844,"$11,521.49"
KAWPOW,5.666999e-07,0.046188,0.001925,$188.16
KHEAVYHASH,4.140686e-02,0.178579,0.007441,$727.48


In [170]:
import numpy as np

max_allowed_coin_market_hp_share = 0.15

reachable_df = pd.DataFrame(index=global_df.index)
reachable_df['algorithm'] = global_df['algorithm']
reachable_df['nethash_eh_s'] = global_df['nethash_eh_s']
reachable_df.loc['BCH', 'nh_hashrate_eh_s'] = nh_limits_df.loc['SHA256ASICBOOST', 'max_hashrate_eh_s']
reachable_df.loc['BTC', 'nh_hashrate_eh_s'] = nh_limits_df.loc['SHA256ASICBOOST', 'max_hashrate_eh_s']
reachable_df.loc['DOGE', 'nh_hashrate_eh_s'] = nh_limits_df.loc['SCRYPT', 'max_hashrate_eh_s']
reachable_df.loc['LTC', 'nh_hashrate_eh_s'] = nh_limits_df.loc['SCRYPT', 'max_hashrate_eh_s']
reachable_df.loc['KAS', 'nh_hashrate_eh_s'] = nh_limits_df.loc['KHEAVYHASH', 'max_hashrate_eh_s']
reachable_df.loc['RVN', 'nh_hashrate_eh_s'] = nh_limits_df.loc['KAWPOW', 'max_hashrate_eh_s']
reachable_df['nh_to_global_ratio'] = reachable_df['nh_hashrate_eh_s'] / reachable_df['nethash_eh_s']  
reachable_df['nh_hashrate_eh_s_capped'] = np.minimum(reachable_df.nh_to_global_ratio, max_allowed_coin_market_hp_share) * reachable_df['nethash_eh_s']

reachable_df

,algorithm,nethash_eh_s,nh_hashrate_eh_s,nh_to_global_ratio,nh_hashrate_eh_s_capped
BCH,SHA-256,2.849742,5.236268e+00,1.837453,4.274613e-01
BTC,SHA-256,752.062414,5.236268e+00,0.006963,5.236268e+00
DOGE,Scrypt,0.002678,9.836617e-05,0.036725,9.836617e-05
KAS,kHeavyHash,1.369009,4.140686e-02,0.030246,4.140686e-02
LTC,Scrypt,0.002323,9.836617e-05,0.042337,9.836617e-05
RVN,KawPow,0.000007,5.666999e-07,0.086789,5.666999e-07


In [175]:
min_order_amount_btc = 0.0001 # BTC
order_duration = 3600  # 1 hour

speed_df = pd.DataFrame(index=nh_market_df.index)
speed_df['paying_btc_eh_s'] = nh_market_df['paying_btc_eh_day'] / 86400
speed_df['speed_eh_s'] = min_order_amount_btc / (speed_df.paying_btc_eh_s * order_duration)
speed_df

,paying_btc_eh_s,speed_eh_s
SCRYPT,0.189050,1.469334e-07
SHA256ASICBOOST,0.000006,4.443380e-03
KAWPOW,0.943335,2.944634e-08
KHEAVYHASH,0.000050,5.564840e-04
